In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
# import xgboost

import sklearn.metrics as metrics


from sklearn.preprocessing import LabelEncoder, StandardScaler

# from imblearn.over_sampling import SMOTE

import warnings

warnings.filterwarnings('ignore')


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.linear_model import LinearRegression
import datetime
import lightgbm

from sklearn import model_selection

from itertools import permutations, combinations
import os

HBox(children=(IntProgress(value=1, bar_style=u'info', max=1), HTML(value=u'')))

In [15]:
def encode(uniques):
    codes = {}
    for i, u in enumerate(uniques):
        codes[u] = i
    return codes

In [3]:
dir_ = '/Users/aakashkerawat/.kaggle/competitions/home-credit-default-risk/'

In [4]:
df = pd.read_csv(dir_+'/application_train.csv.zip', compression='zip')

In [5]:
df_pa = pd.read_csv(dir_+'previous_application.csv.zip', compression='zip')

In [167]:
df_cr = pd.read_csv(dir_+'credit_card_balance.csv.zip', compression='zip')

In [123]:
df_ins = pd.read_csv(dir_+ 'installments_payments.csv.zip', compression='zip')

In [8]:
df_pos = pd.read_csv(dir_+ 'POS_CASH_balance.csv.zip', compression='zip')

In [9]:
df_bb = pd.read_csv(dir_+ 'bureau_balance.csv.zip', compression='zip')

In [10]:
df_bur = pd.read_csv(dir_+'bureau.csv.zip')

In [12]:
df_t = pd.read_csv('/Users/aakashkerawat/.kaggle/competitions/home-credit-default-risk/application_test.csv.zip', compression='zip')

In [ ]:
print 'all files loaded.'

In [16]:
df_c = df.copy()

In [ ]:
del df

In [17]:
df_ct = df_t.copy()

In [ ]:
del df_t

In [ ]:
print 'Label encoding...'

In [18]:
le_dict = {}
for col in df_c:
    if df_c[col].dtype=='O':
        print col
        dict_ = encode(df_c[col].unique())
        df_c[col] = df_c[col].map(dict_)
        df_ct[col] = df_ct[col].map(dict_)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [20]:
def add_features(df_temp):
#     df_temp['EXT_SOURCE_23'] = (df_temp['EXT_SOURCE_2']*df_temp['EXT_SOURCE_3'])

    df_temp['EXT_SOURCE_1_NULL'] = df_temp.EXT_SOURCE_1.isnull()

    df_temp['AGE'] = np.round(np.abs(df_temp['DAYS_BIRTH'])/365)
    
    docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
    live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
    inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

    df_temp['EMPLOYED_AFTER_APP'] = df_temp['DAYS_EMPLOYED']>0
    df_temp['INCOME_CREDIT_RATIO'] = df_temp['AMT_CREDIT']/df_temp['AMT_INCOME_TOTAL']
    df_temp['ANNUITY_UNDER_100K'] = df_temp['AMT_ANNUITY']<100000
    df_temp['AMT_CREDIT_ANNUITY_RATIO'] = df_temp['AMT_CREDIT']/df_temp['AMT_ANNUITY']
    df_temp['AMT_ANNUITY_INCOME_RATIO'] = df_temp['AMT_ANNUITY']/(1+df_temp['AMT_INCOME_TOTAL'])
    
    
#     df_temp['NEW_CREDIT_TO_ANNUITY_RATIO'] = df_temp['AMT_CREDIT'] / df_temp['AMT_ANNUITY']
    df_temp['NEW_CREDIT_TO_GOODS_RATIO'] = df_temp['AMT_CREDIT'] / df_temp['AMT_GOODS_PRICE']
    df_temp['NEW_ANNUITY_TO_GOODS_RATIO'] = df_temp['AMT_ANNUITY'] / df_temp['AMT_GOODS_PRICE']
    df_temp['NEW_DOC_IND_KURT'] = df_temp[docs].kurtosis(axis=1)
    df_temp['NEW_LIVE_IND_SUM'] = df_temp[live].sum(axis=1)
    df_temp['NEW_INC_PER_CHLD'] = df_temp['AMT_INCOME_TOTAL'] / (1 + df_temp['CNT_CHILDREN'])
    df_temp['NEW_INC_BY_ORG'] = df_temp['ORGANIZATION_TYPE'].map(inc_by_org)
    df_temp['NEW_EMPLOY_TO_BIRTH_RATIO'] = df_temp['DAYS_EMPLOYED'] / df_temp['DAYS_BIRTH']
#     df_temp['NEW_ANNUITY_TO_INCOME_RATIO'] = df_temp['AMT_ANNUITY'] / (1 + df_temp['AMT_INCOME_TOTAL'])
    df_temp['NEW_SOURCES_PROD'] = df_temp['EXT_SOURCE_1'] * df_temp['EXT_SOURCE_2'] * df_temp['EXT_SOURCE_3']
    df_temp['NEW_EXT_SOURCES_MEAN'] = df_temp[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    df_temp['NEW_SCORES_STD'] = df_temp[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    df_temp['NEW_SCORES_STD'] = df_temp['NEW_SCORES_STD'].fillna(df_temp['NEW_SCORES_STD'].mean())
    df_temp['NEW_CAR_TO_BIRTH_RATIO'] = df_temp['OWN_CAR_AGE'] / df_temp['DAYS_BIRTH']
    df_temp['NEW_CAR_TO_EMPLOY_RATIO'] = df_temp['OWN_CAR_AGE'] / df_temp['DAYS_EMPLOYED']
    df_temp['NEW_PHONE_TO_BIRTH_RATIO'] = df_temp['DAYS_LAST_PHONE_CHANGE'] / df_temp['DAYS_BIRTH']
    df_temp['NEW_PHONE_TO_BIRTH_RATIO'] = df_temp['DAYS_LAST_PHONE_CHANGE'] / df_temp['DAYS_EMPLOYED']
#     df_temp['NEW_CREDIT_TO_INCOME_RATIO'] = df_temp['AMT_CREDIT'] / df_temp['AMT_INCOME_TOTAL']
    
#     for c in permutations([1,2,3], 2):
#         num, den = c[0], c[1]
#         df_temp['EXT_SOURCE_{}{}'.format(num, den)] = (df_temp['EXT_SOURCE_{}'.format(num)]*df_temp['EXT_SOURCE_{}'.format(den)]).replace(np.inf, np.nan)
    return df_temp

### Bureau data

In [21]:
def get_bureau_features(df_bur_):
    # credit bureau.csv features
# DAYS_CREDIT - min, max, avg
# CREDIT_DAY_OVERDUE - non-zero count, max, min ,mean
# DAYS_CREDIT_ENDDATE - max
# DAYS_ENDDATE_FACT - min
# AMT_CREDIT_MAX_OVERDUE - max
# CNT_CREDIT_PROLONG - max
# AMT_CREDIT_SUM - max, min, avg
# AMT_CREDIT_SUM_DEBT - max
# AMT_CREDIT_SUM_LIMIT - max
# AMT_CREDIT_SUM_OVERDUE - max
# CREDIT_TYPE - unique count
# DAYS_CREDIT_UPDATE - max
# AMT_ANNUITY - mean, max
    
    df_bur_['DAYS_CREDIT_PREV_DIFF'] = df_bur_['DAYS_CREDIT']-df_bur_['DAYS_CREDIT'].shift()

    df_bur_['temp_same_id'] = df_bur_['SK_ID_CURR']!=df_bur_['SK_ID_CURR'].shift()

    df_bur_.loc[df_bur_['temp_same_id']==True, 'DAYS_CREDIT_PREV_DIFF'] = np.nan
    
    df_bur_['DAYS_CREDIT_TENURE'] = df_bur_['DAYS_CREDIT_ENDDATE']-df_bur_['DAYS_CREDIT']

#     del df_bur['temp_same_id']
    df_bur_['AMT_ANNUITY_CREDIT_RATIO'] = (df_bur_['AMT_CREDIT_SUM']/df_bur_['AMT_ANNUITY']).replace(np.inf, np.nan)
    df_bur_['AMT_ANNUITY_CREDIT_OVERDUE_RATIO'] = (df_bur_['AMT_CREDIT_SUM_OVERDUE']/df_bur_['AMT_ANNUITY']).replace(np.inf, np.nan)
    df_bur_['AMT_ANNUITY_CREDIT_DUE_RATIO'] = (df_bur_['AMT_CREDIT_SUM_DEBT']/df_bur_['AMT_ANNUITY']).replace(np.inf, np.nan)
    agg_dict = {'DAYS_CREDIT':['max', 'min', 'mean'], 'CREDIT_DAY_OVERDUE':'max', 'DAYS_CREDIT_ENDDATE':'max',
               'DAYS_ENDDATE_FACT':'min', 'AMT_CREDIT_MAX_OVERDUE':'max', 'CNT_CREDIT_PROLONG':'max',
               'AMT_CREDIT_SUM':['max', 'min', 'mean'], 'AMT_CREDIT_SUM_DEBT':'max', 'AMT_CREDIT_SUM_LIMIT':'max',
               'AMT_CREDIT_SUM_OVERDUE':'max', 'CREDIT_TYPE':'nunique', 'DAYS_CREDIT_UPDATE':'max',
                'AMT_ANNUITY':['max', 'mean'], 
                'AMT_ANNUITY_CREDIT_RATIO':['max', 'mean']
                , 'AMT_ANNUITY_CREDIT_OVERDUE_RATIO':['max', 'mean'] , 'AMT_ANNUITY_CREDIT_DUE_RATIO':['max', 'mean']
               ,'DAYS_CREDIT_PREV_DIFF':['mean', 'max'], 'DAYS_CREDIT_TENURE':['min', 'max']}
    agg_feats = df_bur_.groupby(['SK_ID_CURR']).agg(agg_dict).reset_index()
    agg_feats.columns = ['BUR_'+'_'.join(cols) for cols in agg_feats.columns.values]
    agg_feats.rename(columns={'BUR_SK_ID_CURR_':'SK_ID_CURR'}, inplace=True)
    
    df_bur_['BUR_CREDIT_DAY_OVERDUE_nonzero'] = df_bur_['CREDIT_DAY_OVERDUE']!=0
    cdo = df_bur_.groupby('SK_ID_CURR')['BUR_CREDIT_DAY_OVERDUE_nonzero'].sum().reset_index()
    agg_feats = agg_feats.merge(cdo, 'left', 'SK_ID_CURR')
    


    return agg_feats

In [ ]:
print 'adding bureau features...'

In [22]:
df_bur = df_bur.sort_values(['SK_ID_CURR', 'DAYS_CREDIT', 'SK_ID_BUREAU']).reset_index(drop=True)

In [23]:
ulim = df_bur.loc[df_bur['DAYS_CREDIT_ENDDATE']>5000].index
df_bur = df_bur.drop(ulim).reset_index(drop=True)

In [24]:
df_b = get_bureau_features(df_bur)

In [26]:
df_bur = df_bur.sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop=True)

active_count = df_bur.groupby(['SK_ID_CURR'])['CREDIT_ACTIVE'].value_counts().unstack().reset_index()

active_count = active_count[['SK_ID_CURR', 'Active', 'Sold']]

active_count.fillna(0.0,inplace=True)

active_count = active_count.rename(columns = lambda x: 'BUR_'+x if x!='SK_ID_CURR' else x)

In [27]:
df_b = df_b.merge(active_count, 'left', 'SK_ID_CURR')

### Prev_data

In [29]:
def get_previous_data(df_temp):
    final_agg = df_temp['SK_ID_CURR'].drop_duplicates().reset_index()
    df_temp['DAYS_actual_vs_supposed_first_due'] = df_temp['DAYS_FIRST_DUE']-df_pa['DAYS_LAST_DUE_1ST_VERSION']
    amt_cols = [col for col in df_temp if col.startswith('AMT')]
    days_cols = [col for col in df_temp if col.startswith('DAYS')]
    
    amt_agg = df_temp.groupby('SK_ID_CURR')[amt_cols].agg(['mean', 'max']).reset_index()
    amt_agg.columns = [v[0]+v[1] for v in amt_agg.columns.values]
    final_agg = final_agg.merge(amt_agg, 'left', 'SK_ID_CURR')
    print 'amt agg merged.'
    
    days_agg = df_temp.groupby('SK_ID_CURR')[days_cols].agg(['max', 'min']).reset_index()
    days_agg.columns = [v[0]+v[1] for v in days_agg.columns.values]
    final_agg = final_agg.merge(days_agg, 'left', 'SK_ID_CURR')
    print 'days agg merged.'
    nflag_mean = df_temp.groupby('SK_ID_CURR')['NFLAG_INSURED_ON_APPROVAL'].mean().reset_index().rename(columns={'NFALG_INSURED_ON_APPROVAL':'NFALG_INSURED_ON_APPROVAL_mean'})
    final_agg = final_agg.merge(nflag_mean, 'left', 'SK_ID_CURR')
    
    print 'taking type count'
    type_count_cols = ['NAME_CLIENT_TYPE', 'NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS', 
                       'NAME_YIELD_GROUP', 'NAME_PRODUCT_TYPE'] #'CHANNEL_TYPE'
    for col in type_count_cols:
        print '\t', col
        temp_cnt = df_temp.groupby('SK_ID_CURR')[col].value_counts().unstack().reset_index().fillna(0)
#         print temp_cnt.columns
        temp_cnt = temp_cnt.rename(columns = lambda x: x+'_'+col if x!='SK_ID_CURR' else x)
#         print temp_cnt.columns
        final_agg = final_agg.merge(temp_cnt, 'left', 'SK_ID_CURR')
    
    
#     for i, c in enumerate(permutations(range(len(amt_cols)), 2)):
#         num, den = amt_cols[c[0]], amt_cols[c[1]]
#         df_temp['{}_{}_RATIO'.format(num, den)] = (df_temp['{}'.format(num)]*df_temp['{}'.format(den)]).replace(np.inf, np.nan)
#         print '{}_{}_RATIO'.format(num, den)

#     all ratio permutations can be removed except one or two which might add value (still very little), check f_imp
    
    return final_agg

In [ ]:
print 'adding prev application features...'

In [30]:
size_pa = df_pa.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_PA'})

In [31]:
df_prev = get_previous_data(df_pa)

null_cols = [col for col in df_prev if 'XNA' in col]

df_prev.drop(null_cols, 1, inplace=True)

df_prev = df_prev.merge(size_pa, 'left', 'SK_ID_CURR')

del df_prev['index']

amt agg merged.
days agg merged.
taking type count
	NAME_CLIENT_TYPE
	NAME_CONTRACT_TYPE
	NAME_CONTRACT_STATUS
	NAME_YIELD_GROUP
	NAME_PRODUCT_TYPE


In [32]:
df_prev = df_prev.rename(columns= lambda x: x+'_'+'PREV' if x!='SK_ID_CURR' else x)

### credit balance

In [ ]:
print 'Adding credit balance features...'

In [168]:
df_cr = df_cr.sort_values('SK_ID_CURR').reset_index(drop=True)

In [34]:
def get_credit_features(df_temp):
    amt_cols = [col for col in df_temp if col.startswith('AMT')]
    amt_aggs = df_temp.groupby('SK_ID_CURR')[amt_cols].agg(['mean', 'max']).reset_index()
    amt_aggs.columns = [v[0]+v[1] for v in amt_aggs.columns]
    
    cnt_cols = [col for col in df_temp if col.startswith('CNT')]
    cnt_aggs = df_temp.groupby('SK_ID_CURR')[cnt_cols].agg(['max']).reset_index()
    cnt_aggs.columns = [v[0]+v[1] for v in cnt_aggs.columns]
    
    m_max = df_temp.groupby('SK_ID_CURR')['MONTHS_BALANCE'].idxmax().reset_index()
    
    final_agg = amt_aggs.merge(cnt_aggs, 'outer', 'SK_ID_CURR')
    final_agg = final_agg.merge(m_max, 'left', 'SK_ID_CURR')
    
    return final_agg

In [182]:
def get_credit_features2(df_temp):
    df_temp['LIM_BALANCE_DIFF'] = df_temp['AMT_CREDIT_LIMIT_ACTUAL']-df_temp['AMT_BALANCE']

    df_temp['LIM_BALANCE_DIFF_RATIO'] = (df_temp['LIM_BALANCE_DIFF']/df_temp['AMT_CREDIT_LIMIT_ACTUAL']).replace(np.inf, np.nan)
    agg_dict = {'MONTHS_BALANCE':['min'],
            'AMT_BALANCE':['mean', 'min', 'max'],
            'AMT_CREDIT_LIMIT_ACTUAL':['mean'],
            'AMT_DRAWINGS_ATM_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_DRAWINGS_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_DRAWINGS_OTHER_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_DRAWINGS_POS_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_INST_MIN_REGULARITY':['mean', 'max','sum'],
            'AMT_PAYMENT_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_PAYMENT_TOTAL_CURRENT':['mean', 'max', 'min', 'sum'],
            'AMT_RECEIVABLE_PRINCIPAL':['mean', 'max'],
            'AMT_RECIVABLE':['mean', 'max'],
            'AMT_TOTAL_RECEIVABLE':['mean', 'max'],
            'CNT_DRAWINGS_ATM_CURRENT':['sum', 'mean', 'max'],
            'CNT_DRAWINGS_CURRENT':['sum', 'mean', 'max'],
            'CNT_DRAWINGS_OTHER_CURRENT':['sum', 'mean', 'max'],
            'CNT_DRAWINGS_POS_CURRENT':['sum', 'mean', 'max'],
            'CNT_INSTALMENT_MATURE_CUM':['sum', 'mean', 'max'],
            'SK_DPD':['max', 'sum'],
            'SK_DPD_DEF':['max', 'sum'],
            'LIM_BALANCE_DIFF':['min', 'mean'],
            'LIM_BALANCE_DIFF_RATIO':['mean', 'max', 'min']}
    int_agg = df_temp.groupby('SK_ID_PREV').agg(agg_dict).reset_index()
    int_agg.columns = [v[0]+v[1] for v in int_agg.columns.values]
    
    int_df = df_temp[['SK_ID_CURR', 'SK_ID_PREV']].drop_duplicates()
    
    int_df = int_df.merge(int_agg, 'left', 'SK_ID_PREV')
    final_agg = int_df.groupby('SK_ID_CURR').mean().reset_index()
    
#     final_agg = amt_aggs.merge(cnt_aggs, 'outer', 'SK_ID_CURR')
#     final_agg = final_agg.merge(m_max, 'left', 'SK_ID_CURR')
    
    return final_agg

In [183]:
cr_feats2 = get_credit_features2(df_cr)

In [189]:
size_cr = df_cr.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_CR'})

In [36]:
# cr_feats = get_credit_features(df_cr)

In [37]:
# cr_feats = cr_feats.merge(size_cr, 'left', 'SK_ID_CURR')

In [190]:
cr_feats2 = cr_feats2.merge(size_cr, 'left', 'SK_ID_CURR')

### bureau balance

In [38]:
# res = df_bb.groupby(['SK_ID_BUREAU', 'STATUS'])['MONTHS_BALANCE'].agg(['max', 'min'])

# res = res.unstack()

# res.columns = [v[0]+v[1] for v in res.columns.values]

# res = res.reset_index()

# res_c = df_bb.groupby('SK_ID_BUREAU')['STATUS'].value_counts().unstack().reset_index()

# res_f = res.merge(res_c, 'left', 'SK_ID_BUREAU')

# df_bb_fm = res_f.merge(df_bur[['SK_ID_BUREAU', 'SK_ID_CURR']], 'left', ['SK_ID_BUREAU'])

# df_bb_fm = df_bb_fm.groupby('SK_ID_CURR').agg(['max', 'min'])

# df_bb_fm.columns = [v[0]+v[1] for v in df_bb_fm.columns.values]

# df_bb_fm.reset_index(inplace=True)

# df_bb_fm.drop([col for col in df_bb_fm if col.startswith('SK_ID_BUREAU')], 1, inplace=True)

In [50]:
# df_bb_fm.rename(columns = lambda x: 'BB_'+x if x!='SK_ID_CURR' else x, inplace=True)

### pos cash balance

In [ ]:
print 'adding pos cash balance features...'

In [51]:
df_pos = df_pos.sort_values(['SK_ID_CURR', 'MONTHS_BALANCE']).reset_index(drop=True)
df_size = df_pos.groupby(['SK_ID_CURR', 'MONTHS_BALANCE', 'NAME_CONTRACT_STATUS']).size().reset_index()

size_f = df_size.sort_values(['SK_ID_CURR', 0], ascending=False).drop_duplicates(subset=['SK_ID_CURR', 'NAME_CONTRACT_STATUS'])

ac_com = size_f.pivot('SK_ID_CURR', 'NAME_CONTRACT_STATUS', 0)[['Active', 'Completed']]

ac_com = ac_com.reset_index()

df_pos_fm = ac_com.copy()

df_pos_fm['total'] = df_pos_fm['Active']+df_pos_fm['Completed'].fillna(0)

df_pos_fm.loc[(df_pos_fm['Active'].isnull()) & (df_pos_fm['Completed'].notnull()), 'total'] = -1

df_pos_fm.loc[(df_pos_fm['Active'].isnull()) & (df_pos_fm['Completed'].isnull()), 'total'] = -2

df_pos_fm.rename(columns = lambda x: x+'_aat' if x!='SK_ID_CURR' else x, inplace=True)

n_loans = df_pos.groupby('SK_ID_CURR')['SK_ID_PREV'].nunique().reset_index().rename(columns={'SK_ID_PREV':'total_taken'})

df_pos_fm = df_pos_fm.merge(n_loans, 'left', 'SK_ID_CURR')

com_loans = df_pos.groupby('SK_ID_CURR')['NAME_CONTRACT_STATUS'].value_counts().unstack().reset_index()

df_pos_fm = df_pos_fm.merge(com_loans[['SK_ID_CURR', 'Completed']], 'left', 'SK_ID_CURR')

df_pos_fm.fillna(0, inplace=True)

df_pos_fm['still_active'] = df_pos_fm['total_taken']-df_pos_fm['Completed'].fillna(0)

# avg loan tenure
loan_tenure = df_pos.groupby(['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_FUTURE'].max().reset_index()

loan_tenure = loan_tenure.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].mean().reset_index().rename(columns = {'CNT_INSTALMENT_FUTURE':'avg_loan_tenure'})

df_pos_fm = df_pos_fm.merge(loan_tenure, 'left', 'SK_ID_CURR')

In [52]:
# df_pos = df_pos.sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop=True)

# size_pos = df_pos.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_POS'})

# req_cols = df_pos.columns.drop(['SK_ID_CURR', 'SK_ID_PREV', 'NAME_CONTRACT_STATUS'])

# # mean changed to min after EDA
# df_pos_fm = df_pos.groupby('SK_ID_CURR')[req_cols].agg(['min', 'max']).reset_index()

# df_pos_fm.columns = [v[0]+v[1] for v in df_pos_fm.columns.values]

# df_pos_fm = df_pos_fm.merge(size_pos, 'left', 'SK_ID_CURR')

### installment payment

In [ ]:
print 'adding installment payment features...'

In [125]:
df_ins = df_ins.loc[df_ins['NUM_INSTALMENT_VERSION']!=0].reset_index(drop=True)

In [55]:
def get_ins_features(df_temp):
#     df_temp = df_temp_.copy()
    df_temp['INSTALLMENT_PAYMENT_RATIO'] = (df_temp['AMT_INSTALMENT']/df_temp['AMT_PAYMENT']).replace(np.inf, np.nan)
    df_temp['INSTALLMENT_PAYMENT_DIFF'] = df_temp['AMT_INSTALMENT']-df_temp['AMT_PAYMENT']
    df_ins['DAYS_SUPPOSED_ACTUAL_INS_DIFF'] = df_ins['DAYS_ENTRY_PAYMENT']-df_ins['DAYS_INSTALMENT']
    df_ins['PAYMENT_AFTER_SUPPOSED_DAY'] = ((df_ins['DAYS_SUPPOSED_ACTUAL_INS_DIFF']>0).astype(int))*df_ins['DAYS_SUPPOSED_ACTUAL_INS_DIFF']
    df_ins['PAYMENT_BEFORE_SUPPOSED_DAY'] = ((df_ins['DAYS_SUPPOSED_ACTUAL_INS_DIFF']<0).astype(int))*np.abs(df_ins['DAYS_SUPPOSED_ACTUAL_INS_DIFF'])
    size_ins = df_temp.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_INS'})
    del df_temp['SK_ID_PREV']
    f2 = df_temp.groupby('SK_ID_CURR')['NUM_INSTALMENT_VERSION'].nunique().reset_index().rename(columns={'NUM_INSTALMENT_VERSION':'NUM_INSTALMENT_VERSIONnunique'})
    del df_temp['NUM_INSTALMENT_VERSION']
    f3 = df_temp.groupby('SK_ID_CURR').agg(['mean', 'max', 'min', 'sum']).reset_index()
    f3.columns = [v[0]+v[1] for v in f3.columns.values]
    
    df_ins_fm = f3.merge(f2, 'left', 'SK_ID_CURR')
    df_ins_fm = df_ins_fm.merge(size_ins, 'left', 'SK_ID_CURR')
    return df_ins_fm

In [127]:
def get_ins_features2(df_temp):
#     df_temp = df_temp_.copy()
    df_temp['INSTALLMENT_PAYMENT_RATIO'] = (df_temp['AMT_INSTALMENT']/df_temp['AMT_PAYMENT']).replace(np.inf, np.nan)
    df_temp['INSTALLMENT_PAYMENT_DIFF'] = df_temp['AMT_INSTALMENT']-df_temp['AMT_PAYMENT']
    df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF'] = df_temp['DAYS_ENTRY_PAYMENT']-df_temp['DAYS_INSTALMENT']
    df_temp['PAYMENT_AFTER_SUPPOSED_DAY'] = ((df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF']>0).astype(int))*df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF']
    df_temp['PAYMENT_BEFORE_SUPPOSED_DAY'] = ((df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF']<0).astype(int))*np.abs(df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF'])
    size_ins = df_temp.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_INS'})
    df_temp['DQ_AMOUNT_INSTALMENT_AFTER'] = (df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF']>0).astype(int)*df_temp['INSTALLMENT_PAYMENT_DIFF']
    df_temp['DQ_AMOUNT_INSTALMENT_BEFORE'] = (df_temp['DAYS_SUPPOSED_ACTUAL_INS_DIFF']<0).astype(int)*df_temp['INSTALLMENT_PAYMENT_DIFF']
    
    p_std = df_temp.groupby('SK_ID_PREV')['AMT_PAYMENT'].std().reset_index()
    i_std = df_temp.groupby('SK_ID_PREV')['AMT_INSTALMENT'].std().reset_index()
    i_std = i_std.merge(p_std, 'left', 'SK_ID_PREV')
    i_std['AMT_PAY_INS_STD_DIFF'] = i_std['AMT_INSTALMENT']-i_std['AMT_PAYMENT']
    df_temp = df_temp.merge(i_std, 'left', 'SK_ID_PREV', suffixes=('', '_std'))
#     del df_temp['SK_ID_PREV']
#     f2 = df_temp.groupby('SK_ID_CURR')['NUM_INSTALMENT_VERSION'].nunique().reset_index().rename(columns={'NUM_INSTALMENT_VERSION':'NUM_INSTALMENT_VERSIONnunique'})
#     del df_temp['NUM_INSTALMENT_VERSION']
    
    agg_dict = {'NUM_INSTALMENT_NUMBER':['count'], 
                'DAYS_INSTALMENT': ['min', 'max'],
                'DAYS_ENTRY_PAYMENT':['min', 'max'],
                'AMT_INSTALMENT':['min', 'max', 'sum', 'median'],
                'AMT_PAYMENT':['min', 'max', 'sum', 'median'],
                'INSTALLMENT_PAYMENT_RATIO':['max', 'mean'],
                'INSTALLMENT_PAYMENT_DIFF':['max', 'min','sum', 'mean'], 
                'DAYS_SUPPOSED_ACTUAL_INS_DIFF':['sum'],
                'PAYMENT_AFTER_SUPPOSED_DAY':['mean', 'max', 'min', 'sum'],
                'PAYMENT_BEFORE_SUPPOSED_DAY':['mean', 'max', 'min', 'sum'],
                'AMT_PAY_INS_STD_DIFF':['mean', 'max', 'min'],
                'AMT_PAYMENT_std':['mean', 'max', 'min'],
                'AMT_INSTALMENT_std':['mean', 'max', 'min'],
                'DQ_AMOUNT_INSTALMENT_AFTER':['max', 'mean', 'sum'],
                'DQ_AMOUNT_INSTALMENT_BEFORE':['max', 'mean', 'sum']}
    f1 = df_temp.groupby('SK_ID_CURR').agg(agg_dict).reset_index()
    f1.columns = [v[0]+v[1] for v in f1.columns.values]
    
    df_temp_fm = f1.merge(size_ins, 'left', 'SK_ID_CURR')
    return df_temp_fm

In [128]:
ins_feats = get_ins_features2(df_ins)

In [ ]:
print 'Merging all to train data...'

### merging all data

In [60]:
df_c = add_features(df_c)

In [61]:
# df_c.drop([col for col in df_c if col.startswith('BUR_')], 1, inplace=True)

In [62]:
df_c = df_c.merge(df_b, 'left', 'SK_ID_CURR') # adding bureau features

In [63]:
# df_c.drop([col for col in df_c if col.endswith('_PREV')], 1, inplace=True)

In [64]:
df_c = df_c.merge(df_prev, 'left', 'SK_ID_CURR', suffixes=('', '_PREV'))  # adding prev application features

In [65]:
# df_c = df_c.merge(cr_feats, 'left', 'SK_ID_CURR', suffixes=('', '_CR'))  # adding credit card features

In [192]:
df_c = df_c.merge(cr_feats2, 'left', 'SK_ID_CURR', suffixes=('', '_CR'))  # adding credit card features

In [66]:
# df_c = df_c.merge(df_bb_fm, 'left', 'SK_ID_CURR', suffixes=('', '_BB'))  # adding bureau balance features

In [67]:
df_c = df_c.merge(df_pos_fm, 'left', 'SK_ID_CURR', suffixes=('', '_POS'))  # adding pos cash balance features

In [131]:
df_c = df_c.merge(ins_feats, 'left', 'SK_ID_CURR', suffixes=('', '_INS'))  # adding installment features

In [69]:
bur_size = df_bur.groupby('SK_ID_CURR').size().reset_index().rename(columns={0:'count_bur'})

In [70]:
df_c = df_c.merge(bur_size, 'left', 'SK_ID_CURR')

In [ ]:
print 'merging all to test data...'

In [71]:
df_ct = add_features(df_ct)

In [72]:
df_ct = df_ct.merge(df_b, 'left', 'SK_ID_CURR')

In [73]:
df_ct = df_ct.merge(bur_size, 'left', 'SK_ID_CURR')

In [74]:
df_ct = df_ct.merge(df_prev, 'left', 'SK_ID_CURR', suffixes=('', '_PREV'))

In [75]:
# df_ct = df_ct.merge(cr_feats, 'left', 'SK_ID_CURR', suffixes=('', '_CR'))

In [75]:
df_ct = df_ct.merge(cr_feats2, 'left', 'SK_ID_CURR', suffixes=('', '_CR'))

In [76]:
# df_ct = df_ct.merge(df_bb_fm, 'left', 'SK_ID_CURR', suffixes=('', '_BB'))  # adding bureau balance features

In [77]:
df_ct = df_ct.merge(df_pos_fm, 'left', 'SK_ID_CURR', suffixes=('', '_POS'))  # adding pos cash balance features

In [78]:
df_ct = df_ct.merge(ins_feats, 'left', 'SK_ID_CURR', suffixes=('', '_INS'))  # adding installment features

### modeling

In [1]:
predictors = df_c.columns.drop(['SK_ID_CURR', 'TARGET', 'DAYS_BIRTH', 'NEW_EXT_SOURCES_MEAN', 'NEW_SOURCES_PROD'])

NameError: name 'df_c' is not defined

In [ ]:
# predictors_wr = df_c.columns.drop(['SK_ID_CURR', 'TARGET', 'DAYS_BIRTH']+[col for col in df_c if col.endswith('RATIO_PREV_D')])

In [194]:
print 'total features - {}'.format(len(predictors))

326

In [398]:
# lgb = lightgbm.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=6)

# lgb = lightgbm.LGBMClassifier(n_estimators=1000, learning_rate=0.02, max_depth=7, reg_lambda=15, reg_alpha=10,feature_fraction=0.6)

# lgb = lightgbm.LGBMClassifier(n_estimators=2500, learning_rate=0.015, max_depth=8, reg_lambda=20, reg_alpha=10, feature_fraction=0.6)

# # curr best
# lgb = lightgbm.LGBMClassifier(n_estimators=2500, learning_rate=0.02, max_depth=8, reg_lambda=20, reg_alpha=10, feature_fraction=0.6, subsample=0.9, random_state=2018)

In [195]:
# curr best 2
lgb = lightgbm.LGBMClassifier(n_estimators=3000, learning_rate=0.01, max_depth=8, reg_lambda=25, reg_alpha=10, feature_fraction=0.6, subsample=0.9, random_state=2018)

In [158]:
# lgb = lightgbm.LGBMClassifier(n_estimators=3000, learning_rate=0.015, max_depth=8, reg_lambda=25, reg_alpha=10, feature_fraction=0.6, subsample=0.7, random_state=2018)

In [85]:
def run_cv(clf, df_temp, predictors_, target, metric_func, predict_proba=False, shuffle_=False, max_run=1):
    kf = model_selection.KFold(shuffle=shuffle_, random_state=2018)
    all_scores = []
    all_scores_tr = []
    count = 1
    for train_i, cv_i in kf.split(df_temp):
        df_tr = df_temp.loc[train_i]
#         org_type = df_tr.groupby('ORGANIZATION_TYPE')['TARGET'].mean()
        
#         df_tr = df_tr.drop(df_tr.loc[df_tr['DAYS_EMPLOYED']>0].index).reset_index(drop=True)
        df_cv = df_temp.loc[cv_i]
#         df_cv['ORGANIZATION_TYPE'] = df_cv['ORGANIZATION_TYPE'].map(org_type)
#         print df_cv['ORGANIZATION_TYPE'].head()
        clf.fit(df_tr[predictors_], df_tr[target])
#         df_temp.loc[cv_i, 'MODEL_0'] = clf.predict_proba(df_cv[predictors_])[:,1]
        if predict_proba:
            preds = clf.predict_proba(df_cv[predictors_])[:,1]
            preds_tr = clf.predict_proba(df_tr[predictors_])[:,1]
        else:
            preds = clf.predict(df_cv[predictors_])
            preds = clf.predict(df_tr[predictors_])
        score = metric_func(df_cv[target], preds)
        score_tr = metric_func(df_tr[target], preds_tr)
        print 'Test score - ', score
        print 'Train score - ', score_tr, '\n'
        all_scores.append(score)
        all_scores_tr.append(score_tr)
        if (count==max_run) and (max_run>0):
            break
        count+=1
    print 'Mean test score =', np.mean(all_scores)
    print 'Mean train score =', np.mean(all_scores_tr)
    return clf, df_temp

In [161]:
clf, df_c = run_cv(lgb, df_c, predictors, 'TARGET', metrics.roc_auc_score, predict_proba=True, max_run=1)

Test score -  0.7903906843211165
Train score -  0.8648447609859049 

Mean test score = 0.7903906843211165
Mean train score = 0.8648447609859049


In [162]:
f_imp = pd.Series(clf.booster_.feature_importance(importance_type='gain'), predictors).sort_values(ascending=False)

In [ ]:
print 'Feature importance...'

In [ ]:
print f_imp[:170]